In [1]:
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [2]:
 !pip install sklearn-genetic
 !pip install scikit-learn
 !pip install vecstack
 !pip install mlxtend

In [3]:
!pip install sklearn-genetic-opt

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import imblearn
from imblearn.under_sampling import RandomUnderSampler 

from genetic_selection import GeneticSelectionCV
from sklearn_genetic import GASearchCV
from sklearn_genetic.space import Continuous, Categorical, Integer
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

In [2]:
train_data = pd.read_csv('/content/drive/My Drive/Santander Customer Satisfaction - TRAIN.csv')

test_data = pd.read_csv('/content/drive/My Drive/Santander Customer Satisfaction - TEST-Without TARGET.csv')

train_data


,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38,TARGET
0,1,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39205.170,0
1,3,2,34,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,1,1,1,0,0,0,0,1,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,300.0,122.22,300.0,240.75,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,49278.030,0
2,4,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.00,2.07,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,67333.770,0
3,8,2,37,0.0,195.0,195.0,0.0,0.0,0.0,0.0,0.0,195.0,195.0,0.0,0.0,195.0,0.0,0.0,195.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,91.56,138.84,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,64007.970,0
4,10,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,40501.08,13501.47,0.0,0.0,0.0,0.0,0.00,0.0,85501.89,85501.89,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117310.979,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76015,151829,2,48,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60926.490,0
76016,151830,2,39,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,...,0.00,0.00,0.0,0.0,0.0,0.0,15498.42,0.0,48175.62,31837.02,0.0,0.00,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0

In [3]:
test_data

,ID,var3,var15,imp_ent_var16_ult1,imp_op_var39_comer_ult1,imp_op_var39_comer_ult3,imp_op_var40_comer_ult1,imp_op_var40_comer_ult3,imp_op_var40_efect_ult1,imp_op_var40_efect_ult3,imp_op_var40_ult1,imp_op_var41_comer_ult1,imp_op_var41_comer_ult3,imp_op_var41_efect_ult1,imp_op_var41_efect_ult3,imp_op_var41_ult1,imp_op_var39_efect_ult1,imp_op_var39_efect_ult3,imp_op_var39_ult1,imp_sal_var16_ult1,ind_var1_0,ind_var1,ind_var2_0,ind_var2,ind_var5_0,ind_var5,ind_var6_0,ind_var6,ind_var8_0,ind_var8,ind_var12_0,ind_var12,ind_var13_0,ind_var13_corto_0,ind_var13_corto,ind_var13_largo_0,ind_var13_largo,ind_var13_medio_0,ind_var13_medio,ind_var13,...,saldo_medio_var5_hace3,saldo_medio_var5_ult1,saldo_medio_var5_ult3,saldo_medio_var8_hace2,saldo_medio_var8_hace3,saldo_medio_var8_ult1,saldo_medio_var8_ult3,saldo_medio_var12_hace2,saldo_medio_var12_hace3,saldo_medio_var12_ult1,saldo_medio_var12_ult3,saldo_medio_var13_corto_hace2,saldo_medio_var13_corto_hace3,saldo_medio_var13_corto_ult1,saldo_medio_var13_corto_ult3,saldo_medio_var13_largo_hace2,saldo_medio_var13_largo_hace3,saldo_medio_var13_largo_ult1,saldo_medio_var13_largo_ult3,saldo_medio_var13_medio_hace2,saldo_medio_var13_medio_hace3,saldo_medio_var13_medio_ult1,saldo_medio_var13_medio_ult3,saldo_medio_var17_hace2,saldo_medio_var17_hace3,saldo_medio_var17_ult1,saldo_medio_var17_ult3,saldo_medio_var29_hace2,saldo_medio_var29_hace3,saldo_medio_var29_ult1,saldo_medio_var29_ult3,saldo_medio_var33_hace2,saldo_medio_var33_hace3,saldo_medio_var33_ult1,saldo_medio_var33_ult3,saldo_medio_var44_hace2,saldo_medio_var44_hace3,saldo_medio_var44_ult1,saldo_medio_var44_ult3,var38
0,2,2,32,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.43,6.00,4.80,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40532.100
1,5,2,35,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,2.55,3.00,2.85,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,45486.720
2,6,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,60.0,60.0,60.0,60.0,60.0,60.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,57.00,51.45,66.15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,46993.950
3,7,2,24,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,187898.610
4,9,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3.87,30.00,21.30,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,73649.730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75813,151831,2,23,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,40243.200
75814,151832,2,26,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,1.23,3.00,2.40,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,146961.300
75815,1

In [5]:
x_train = train_data.drop(columns = [ 'ID' , 'TARGET'])
y_train = train_data['TARGET']

x_test = test_data.drop(columns = 'ID')

In [6]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)

(76020, 369)
(76020,)
(75818, 369)


In [7]:
rus = RandomUnderSampler(sampling_strategy = 0.5 , random_state=42)
x_res, y_res = rus.fit_resample(x_train, y_train)

In [12]:
# Random Forest Classifier----------Top 25

rfc = RandomForestClassifier()

model = GeneticSelectionCV(rfc,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=25,
                                  n_population=30,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=10,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=1)

model = model.fit(x_res, y_res)

features = model.get_support(indices = True)

print(features)

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                    	max                      
0  	30    	[ 0.67118836 13.23333333]	[0.01468965 7.41926921]	[0.63696802 1.        ]	[ 0.69658831 25.        ]
1  	23    	[-2332.81261758    20.83333333]	[4229.81311388    9.31694991]	[-1.e+04  5.e+00]      	[ 0.69658831 42.        ]
2  	19    	[-1999.45358959    22.56666667]	[4000.27320522    7.45512501]	[-10000.     12.]      	[ 0.69658831 44.        ]
3  	18    	[-1666.09355337    22.93333333]	[3727.03626656    6.86019112]	[-10000.     13.]      	[ 0.69658831 42.        ]
4  	16    	[-1332.73343556    23.6       ]	[3399.58164166    7.12086137]	[-10000.     15.]      	[ 0.69769616 44.        ]
5  	21    	[-2332.80222643    25.3       ]	[4229.81884643    7.17007671]	[-10000.     18.]      	[ 0.7005773 44.       ]  
6  	17    	[-1332.72943895    23.5       ]	[3399.58320926    5.34009987]	[-10000.     19.]      	[ 0.70456664 41.       

In [13]:
### Hyperparameter Tuning for the model using GA---Top 25--rfc

param = {'n_estimators' : Integer(100 , 200) ,'max_depth'  : Integer(5,7)}

rfc = RandomForestClassifier()
new_model = GASearchCV(estimator=rfc,
                               cv= 3,
                               scoring='accuracy',
                               population_size=10,
                               generations=2,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

rfc_25 = x_res.iloc[ : , [15,24,33,35,52,97,98,112,119,137,146,170,221,236,275,289,311,332,336,352]]

new_model = new_model.fit(rfc_25, y_res)

print(new_model.best_params_)
print(new_model.best_score_)


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.687844	0.00445044 	0.694038   	0.681405   
1  	16    	0.694005	0.00383142 	0.698914   	0.685284   
2  	20    	0.696953	0.0026981  	0.699357   	0.692265   
{'n_estimators': 102, 'max_depth': 7}
0.6993572695035462


In [15]:
# fitting the dataset--Top 25--rfc
rfc = RandomForestClassifier(max_depth=7, n_estimators=102)

x_train1 = x_train.iloc[ : , [15,24,33,35,52,97,98,112,119,137,146,170,221,236,275,289,311,332,336,352]]
x_test1 =  test_data.iloc[ : , [15,24,33,35,52,97,98,112,119,137,146,170,221,236,275,289,311,332,336,352]]


rfc = rfc.fit( x_train1 ,y_train)
pred = pd.DataFrame(rfc.predict(x_test1))
pred

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- imp_amort_var18_hace3
- imp_op_var41_ult1
- imp_reemb_var13_ult1
- ind_var13_corto_0
- ind_var13_largo_0
- ...
Feature names seen at fit time, yet now missing:
- imp_amort_var18_ult1
- imp_op_var39_efect_ult1
- imp_reemb_var17_hace3
- ind_var13_corto
- ind_var13_largo
- ...

  warnings.warn(message, FutureWarning)


,0
0,0
1,0
2,0
3,0
4,0
...,...
75813,0
75814,0
75815,0
75816,0


In [16]:
#Kaggle Submission
pred.to_csv((r'/content/drive/MyDrive/Colab Notebooks/rfc_25.csv'), index=False)

In [12]:
# Random Forest Classifier----------Top 50

rfc = RandomForestClassifier()

model = GeneticSelectionCV(rfc,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=50,
                                  n_population=10,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=20,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=5,
                                  caching=True,
                                  n_jobs=-1)

model = model.fit(x_res, y_res)

features = model.get_support(indices = True)

print(features)

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                      	min                    	max                      
0  	10    	[ 0.67165413 16.2       ]	[ 0.01629915 13.84052022]	[0.65192819 4.        ]	[ 0.70124353 48.        ]
1  	2     	[ 0.68776764 25.9       ]	[1.17773595e-02 1.48421697e+01]	[0.66389716 4.        ]	[ 0.70124353 48.        ]
2  	8     	[-3999.58374229    47.9       ]	[4899.31935857   10.03444069]  	[-10000.     33.]      	[ 0.70124353 64.        ]
3  	0     	[-1999.44099999    40.6       ]	[4000.2795        7.56571213]  	[-10000.     33.]      	[ 0.70124353 55.        ]
4  	4     	[ 0.70154276 37.        ]      	[0.00047296 0.        ]        	[ 0.70124353 37.        ]	[ 0.70268422 37.        ]
5  	6     	[-2999.50886361    42.2       ]	[4582.89721919    8.09691299]  	[-10000.     37.]        	[ 0.70268422 58.        ]
[  7  14  23  40  42  44  46  55  61  70  75  98 120 126 148 155 166 170
 178 183 190 200 210 217 251 254 25

In [13]:
# Hyperparameter Tuning for the model using GA---Top 50--rfc

param = {'n_estimators' : Integer(100 , 200) ,'max_depth'  : Integer(5,7)}

rfc = RandomForestClassifier()

new_model = GASearchCV(estimator=rfc,
                               cv= 3,
                               scoring='accuracy',
                               population_size=10,
                               generations=2,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

rfc_50 = x_res.iloc[ : , [7,14,23,40,42,44,46,55,61,70,75,98,120,126,148,155,166,170,
178,183,190,200,210,217,251,254,259,274,288,307,330,335,337,343,344,359,363]]

new_model = new_model.fit(rfc_50, y_res)

print(new_model.best_params_)
print(new_model.best_score_)


/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.694537	0.00876414 	0.706339   	0.683067   
1  	16    	0.69652 	0.00630699 	0.707114   	0.689495   
2  	18    	0.69611 	0.00658771 	0.707114   	0.689273   
{'n_estimators': 180, 'max_depth': 7}
0.7071143617021277


In [18]:
# fitting the dataset--Top 50--rfc
rfc = RandomForestClassifier(max_depth=7, n_estimators=180)

x_train1 = x_train.iloc[ : , [7,14,23,40,42,44,46,55,61,70,75,98,120,126,148,155,166,170,
178,183,190,200,210,217,251,254,259,274,288,307,330,335,337,343,344,359,363]]
x_test1 =  test_data.iloc[ : , [7,14,23,40,42,44,46,55,61,70,75,98,120,126,148,155,166,170,
178,183,190,200,210,217,251,254,259,274,288,307,330,335,337,343,344,359,363]]


rfc = rfc.fit( x_train1 ,y_train)
pred1 = pd.DataFrame(rfc.predict(x_test1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- delta_imp_compra_var44_1y3
- delta_num_aport_var17_1y3
- delta_num_trasp_var17_out_1y3
- imp_op_var40_comer_ult3
- imp_op_var41_efect_ult3
- ...
Feature names seen at fit time, yet now missing:
- delta_imp_reemb_var13_1y3
- delta_num_aport_var33_1y3
- delta_num_trasp_var33_in_1y3
- imp_op_var40_efect_ult1
- imp_op_var41_ult1
- ...

  warnings.warn(message, FutureWarning)


In [19]:
#Kaggle Submission
pred1.to_csv((r'/content/drive/MyDrive/Colab Notebooks/rfc_50.csv'), index=False)

In [9]:
# Gradient Boosting Classifier----------Top 25

gbc = GradientBoostingClassifier()

model = GeneticSelectionCV(gbc, 
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=25,
                                  n_population=10,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=30,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=5,
                                  caching=True,
                                  n_jobs=-1)

gbc = model.fit(x_res, y_res)

features = gbc.get_support(indices = True)

print(features)

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                    	min                    	max                      
0  	10    	[ 0.66979175 10.4       ]	[0.0100876  7.39188745]	[0.66334203 1.        ]	[ 0.69924882 22.        ]
1  	7     	[ 0.6808961 10.7      ]  	[0.01427219 5.93380148]	[0.66644502 2.        ]	[ 0.69924882 21.        ]
2  	6     	[-2999.51125729    16.6       ]	[4582.89565217    8.36899038]	[-1.e+04  9.e+00]      	[ 0.69924882 31.        ]
3  	4     	[-999.37141868   15.9       ]  	[3000.20952711    8.33606622]	[-1.e+04  9.e+00]      	[ 0.70080001 35.        ]
4  	8     	[-2999.50983882    23.2       ]	[4582.89658077    9.76524449]	[-1.e+04  9.e+00]      	[ 0.70611918 41.        ]
5  	8     	[-2999.50751179    27.3       ]	[4582.89810417    7.56372924]	[-10000.     20.]      	[ 0.70611918 43.        ]
6  	5     	[-999.3645703   25.6      ]    	[3000.2118099    7.8      ]  	[-10000.     23.]      	[ 0.70611918 49.        ]
7  	4    

In [ ]:
# Hyperparameter Tuning for the model using GA---Top 25--gbc

param = {'n_estimators' : Integer(100 , 200) ,'max_depth'  : Integer(5,7)}

gbc = GradientBoostingClassifier()

new_model = GASearchCV(estimator=gbc,
                               cv= 3,
                               scoring='accuracy',
                               population_size=10,
                               generations=2,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

gbc_25 = x_res.iloc[ : , [44,71,75,88,160,161,163,195,199,200,220,225,239,244,252,277,284,310,317,331,341,349,365]]

new_model = new_model.fit(gbc_25, y_res)

print(new_model.best_params_)
print(new_model.best_score_)

gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.709608	0.00229538 	0.713209   	0.705785   
1  	14    	0.711625	0.00184304 	0.713209   	0.708112   
2  	19    	0.712677	0.000588054	0.713209   	0.711547   


{'max_depth': 6, 'n_estimators': 121}

In [10]:
# fitting the dataset--Top 25--gbc
gbc = GradientBoostingClassifier(max_depth=6, n_estimators=121)

x_train1 = x_train.iloc[ : , [44,71,75,88,160,161,163,195,199,200,220,225,239,244,252,277,284,310,317,331,341,349,365]]
x_test1 =  test_data.iloc[ : , [44,71,75,88,160,161,163,195,199,200,220,225,239,244,252,277,284,310,317,331,341,349,365]]


gbc = gbc.fit( x_train1 ,y_train)
pred2 = pd.DataFrame(gbc.predict(x_test1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- delta_imp_amort_var18_1y3
- delta_imp_aport_var33_1y3
- delta_num_venta_var44_1y3
- imp_aport_var13_hace3
- imp_reemb_var33_hace3
- ...
Feature names seen at fit time, yet now missing:
- delta_imp_amort_var34_1y3
- delta_imp_reemb_var13_1y3
- imp_amort_var18_hace3
- imp_aport_var13_ult1
- imp_reemb_var33_ult1
- ...

  warnings.warn(message, FutureWarning)


In [11]:
#Kaggle Submission
pred2.to_csv((r'/content/drive/MyDrive/Colab Notebooks/gbc_25.csv'), index=False)

In [8]:
# Gradient Boosting Classifier----------Top 50

gbc = GradientBoostingClassifier()

model = GeneticSelectionCV(gbc,
                                  verbose=1,
                                  scoring="accuracy",
                                  max_features=50,
                                  n_population=30,
                                  crossover_proba=0.5,
                                  mutation_proba=0.2,
                                  n_generations=10,
                                  crossover_independent_proba=0.5,
                                  mutation_independent_proba=0.05,
                                  tournament_size=3,
                                  n_gen_no_change=10,
                                  caching=True,
                                  n_jobs=-1)

model = model.fit(x_res, y_res)

features = model.get_support(indices = True)

print(features)

Selecting features with genetic algorithm.
gen	nevals	avg                      	std                      	min                    	max                      
0  	30    	[ 0.69438641 25.6       ]	[ 0.02726799 14.28892345]	[0.66378463 1.        ]	[ 0.77028008 49.        ]
1  	16    	[-1332.71194089    38.36666667]	[3399.59007266   14.68896033]	[-1.e+04  4.e+00]      	[ 0.77559944 72.        ]
2  	23    	[-1332.68697809    42.4       ]	[3399.59986388    8.87543426]	[-10000.     27.]      	[ 0.78047528 63.        ]
3  	19    	[-999.31067805   41.46666667]  	[3000.22977401    8.99530742]	[-10000.     25.]      	[ 0.78047528 57.        ]
4  	13    	[-2666.09955967    48.4       ]	[4422.50836553    9.45374705]	[-10000.     25.]      	[ 0.78047528 68.        ]
5  	15    	[-2666.09780513    47.73333333]	[4422.50942356    9.19033309]	[-10000.     25.]      	[ 0.78047528 68.        ]
6  	18    	[-1999.37749267    48.66666667]	[4000.31125366    7.23571389]	[-10000.     29.]      	[ 0.78069688 64.   

In [9]:
# Hyperparameter Tuning for the model using GA---Top 50--gbc

param = {'n_estimators' : Integer(100 , 200) ,'max_depth'  : Integer(5,7)}

gbc = GradientBoostingClassifier()

new_model = GASearchCV(estimator=gbc,
                               cv= 3,
                               scoring='accuracy',
                               population_size=10,
                               generations=2,
                               tournament_size=3,
                               elitism=True,
                               crossover_probability=0.8,
                               mutation_probability=0.1,
                               param_grid=param,
                               criteria='max',
                               algorithm='eaMuPlusLambda',
                               n_jobs=-1,
                               verbose=True,
                               keep_top_k=4)

gbc_50 = x_res.iloc[ : , [ 1,14,19,  23,  27,  28,  32,  38, 39,  50,  55, 56,  73,  82,  83,  87, 110, 118,
 121, 132, 134, 135, 137, 138, 150, 151, 176, 180, 182, 197, 212, 223, 233, 234, 241, 274,
 275, 295, 299, 311, 328, 334, 336, 342, 344, 345, 355, 357, 365]]

new_model = new_model.fit(gbc_50, y_res)

print(new_model.best_params_)
print(new_model.best_score_)

/usr/local/lib/python3.7/dist-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	fitness 	fitness_std	fitness_max	fitness_min
0  	10    	0.774091	0.00103029 	0.775155   	0.771941   
1  	20    	0.775044	0.000788272	0.776263   	0.773493   
2  	17    	0.776363	0.000793629	0.777371   	0.775044   
{'n_estimators': 101, 'max_depth': 6}
0.7773714539007092


In [10]:
# fitting the dataset--Top 50--gbc
gbc = GradientBoostingClassifier(max_depth=6, n_estimators=101)

x_train1 = x_train.iloc[ : , [ 1,14,19,  23,  27,  28,  32,  38, 39,  50,  55, 56,  73,  82,  83,  87, 110, 118,
 121, 132, 134, 135, 137, 138, 150, 151, 176, 180, 182, 197, 212, 223, 233, 234, 241, 274,
 275, 295, 299, 311, 328, 334, 336, 342, 344, 345, 355, 357, 365]]
x_test1 =  test_data.iloc[ : , [1,14,19,  23,  27,  28,  32,  38, 39,  50,  55, 56,  73,  82,  83,  87, 110, 118,
 121, 132, 134, 135, 137, 138, 150, 151, 176, 180, 182, 197, 212, 223, 233, 234, 241, 274,
 275, 295, 299, 311, 328, 334, 336, 342, 344, 345, 355, 357, 365]]


gbc = gbc.fit( x_train1 ,y_train)
pred3 = pd.DataFrame(gbc.predict(x_test1))

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:488: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names unseen at fit time:
- delta_imp_aport_var13_1y3
- delta_num_compra_var44_1y3
- imp_amort_var34_hace3
- imp_compra_var44_hace3
- imp_op_var41_efect_ult3
- ...
Feature names seen at fit time, yet now missing:
- delta_imp_aport_var17_1y3
- delta_num_reemb_var13_1y3
- imp_amort_var34_ult1
- imp_op_var41_ult1
- imp_reemb_var13_hace3
- ...

  warnings.warn(message, FutureWarning)


In [11]:
#Kaggle Submission
pred3.to_csv((r'/content/drive/MyDrive/Colab Notebooks/gbc_50.csv'), index=False)

In [13]:
# SelectKBest
#TOP-----25
model = SelectKBest(score_func=f_classif , k=25)

model_final = model.fit_transform(x_res,y_res)

features_25 = model.get_support(indices=True)
print( 'top 25 : ', feature_names_25)
print('\n')
#TOP-------50
model = SelectKBest(score_func=f_classif , k=50)

model_final = model.fit_transform(x_res,y_res)

features_50 = model.get_support(indices=True)
print( 'top 50 :' ,feature_names_50)



top 25 :  [  1  24  27  29  30  31  32  33  38  63  88  90  93  96  97  98  99 104
 138 147 158 182 193 280 283]


top 50 : [  1  12  13  14  15  16  17  23  24  27  28  29  30  31  32  33  38  48
  49  63  76  79  88  90  93  94  96  97  98  99 104 113 114 138 147 151
 154 158 168 171 182 190 193 263 280 282 283 340 342 343]


/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 21  22  25  26  36  37  43  44  56  57  58  59  60  61  71  72  80  84
  85  91  92 102 103 109 110 131 132 133 134 135 136 145 146 155 161 162
 165 170 174 179 180 181 186 189 192 194 195 198 202 203 204 205 206 210
 214 215 216 217 218 220 221 222 223 228 229 230 234 236 238 239 242 243
 244 245 246 247 248 249 252 261 262 267 268 269 285 287 303 305 307 308
 313 314 315 316 317 318 319 320 327 348 349 350 351 356 357 358 359 361] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:113: RuntimeWarning: invalid value encountered in true_divide
  f = msb / msw
/usr/local/lib/python3.7/dist-packages/sklearn/feature_selection/_univariate_selection.py:112: UserWarning: Features [ 21  22  25  26  36  37  43  44  56  57  58  59  60  61  71  7